In [1]:
import numpy as np
import warnings
import sys
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter("ignore")


In [2]:
def get_data(path, noise=False):
    data = np.load(path)
    if noise == True :
        data = data + np.random.normal(0, 0.001, data.shape)
    return data

In [3]:
X_train = get_data('enron-train-features.pkl').astype(np.uint8)
y_train = get_data('enron-train-labels.pkl').astype(np.int8)
X_test = get_data('enron-test-features.pkl').astype(np.uint8)
y_test = get_data('enron-test-labels.pkl').astype(np.int8)

In [4]:
print("Train_X: ",X_train.shape)
print("Train_Y: ",y_train.shape)
print("Test_X: ",X_test.shape)
print("Test_Y: ",y_test.shape)

Train_X:  (1123, 1001)
Train_Y:  (1123, 53)
Test_X:  (579, 1001)
Test_Y:  (579, 53)


In [5]:
def hamming_accuracy_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

## Chain Model type 1


In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.backend.tensorflow_backend import clear_session

Using TensorFlow backend.


In [7]:
def create_c1node(X_feed,y_now):
    '''
    C1 node Architecture:
    attribute:512:256:1 [saperate for each class]
    loss: Binary crossentropy
    '''
    model = Sequential()
    model.add(Dense(512,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='glorot_uniform'))
    model.add(Dropout(0.4))
    model.add(Dense(256,activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Dropout(0.4))
    model.add(Dense(y_now.shape[1],activation='sigmoid',kernel_initializer='glorot_uniform'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    
    return model

In [8]:
#Model chain
chain = []

#Training
X_feed = X_train.copy()
for i in range(y_train.shape[1]):
    print("Training chain node ",i)
    y_now = y_train[:,[i,]].copy()
    print("Shapes:\n X = {} \n Y = {}".format(X_feed.shape,y_now.shape))

    node = create_c1node(X_feed,y_now)
    node.fit(X_feed,y_now,epochs=50,batch_size=50)
    print("Training of node {} complete\n\n".format(i))  
    #Checking accuracy of the node
    #---
    #Adding node to chain 
    chain.append(node)
    #Updating X_feed
    X_feed = np.append(X_feed,y_now,axis=1)


Training chain node  0
Shapes:
 X = (1123, 1001) 
 Y = (1123, 1)
Epoch 1/50
1123/1123 [==============================] - 10s 9ms/step - loss: 0.1948 - acc: 0.9688
Epoch 2/50
1123/1123 [==============================] - 0s 227us/step - loss: 0.1298 - acc: 0.9840
Epoch 3/50
1123/1123 [==============================] - 0s 224us/step - loss: 0.0942 - acc: 0.9840
Epoch 4/50
1123/1123 [==============================] - 0s 229us/step - loss: 0.0843 - acc: 0.9840
Epoch 5/50
1123/1123 [==============================] - 0s 241us/step - loss: 0.0706 - acc: 0.9840
Epoch 6/50
1123/1123 [==============================] - 0s 226us/step - loss: 0.0610 - acc: 0.9840
Epoch 7/50
1123/1123 [==============================] - 0s 231us/step - loss: 0.0539 - acc: 0.9840
Epoch 8/50
1123/1123 [==============================] - 0s 228us/step - loss: 0.0525 - acc: 0.9840
Epoch 9/50
1123/1123 [==============================] - 0s 229us/step - loss: 0.0475 - acc: 0.9884
Epoch 10/50
1123/1123 [======================

1123/1123 [==============================] - 0s 229us/step - loss: 0.0107 - acc: 0.9973
Epoch 32/50
1123/1123 [==============================] - 0s 228us/step - loss: 0.0044 - acc: 0.9973
Epoch 33/50
1123/1123 [==============================] - 0s 227us/step - loss: 0.0069 - acc: 0.9973
Epoch 34/50
1123/1123 [==============================] - 0s 226us/step - loss: 0.0016 - acc: 1.0000
Epoch 35/50
1123/1123 [==============================] - 0s 224us/step - loss: 0.0063 - acc: 0.9973
Epoch 36/50
1123/1123 [==============================] - 0s 222us/step - loss: 0.0028 - acc: 0.9991
Epoch 37/50
1123/1123 [==============================] - 0s 224us/step - loss: 0.0066 - acc: 0.9973
Epoch 38/50
1123/1123 [==============================] - 0s 222us/step - loss: 0.0055 - acc: 0.9964
Epoch 39/50
1123/1123 [==============================] - 0s 224us/step - loss: 0.0048 - acc: 0.9982
Epoch 40/50
1123/1123 [==============================] - 0s 222us/step - loss: 0.0055 - acc: 0.9973
Epoch 41/50


1123/1123 [==============================] - 0s 227us/step - loss: 0.0094 - acc: 0.9973
Epoch 12/50
1123/1123 [==============================] - 0s 224us/step - loss: 0.0034 - acc: 0.9991
Epoch 13/50
1123/1123 [==============================] - 0s 223us/step - loss: 0.0041 - acc: 0.9982
Epoch 14/50
1123/1123 [==============================] - 0s 227us/step - loss: 0.0044 - acc: 0.9991
Epoch 15/50
1123/1123 [==============================] - 0s 221us/step - loss: 0.0080 - acc: 0.9982
Epoch 16/50
1123/1123 [==============================] - 0s 223us/step - loss: 0.0038 - acc: 0.9991
Epoch 17/50
1123/1123 [==============================] - 0s 221us/step - loss: 0.0037 - acc: 0.9991
Epoch 18/50
1123/1123 [==============================] - 0s 223us/step - loss: 0.0050 - acc: 0.9991
Epoch 19/50
1123/1123 [==============================] - 0s 227us/step - loss: 0.0023 - acc: 0.9991
Epoch 20/50
1123/1123 [==============================] - 0s 227us/step - loss: 9.5984e-04 - acc: 1.0000
Epoch 21

1123/1123 [==============================] - 0s 228us/step - loss: 0.0376 - acc: 0.9938
Epoch 42/50
1123/1123 [==============================] - 0s 226us/step - loss: 0.0455 - acc: 0.9893
Epoch 43/50
1123/1123 [==============================] - 0s 229us/step - loss: 0.0413 - acc: 0.9911
Epoch 44/50
1123/1123 [==============================] - 0s 226us/step - loss: 0.0428 - acc: 0.9902
Epoch 45/50
1123/1123 [==============================] - 0s 226us/step - loss: 0.0388 - acc: 0.9929 0s - loss: 0.0325 - acc: 0.99
Epoch 46/50
1123/1123 [==============================] - 0s 224us/step - loss: 0.0256 - acc: 0.9947
Epoch 47/50
1123/1123 [==============================] - 0s 222us/step - loss: 0.0482 - acc: 0.9893
Epoch 48/50
1123/1123 [==============================] - 0s 224us/step - loss: 0.0377 - acc: 0.9955
Epoch 49/50
1123/1123 [==============================] - 0s 225us/step - loss: 0.0423 - acc: 0.9893
Epoch 50/50
1123/1123 [==============================] - 0s 226us/step - loss: 0.0

1123/1123 [==============================] - 0s 231us/step - loss: 0.0841 - acc: 0.9653
Epoch 21/50
1123/1123 [==============================] - 0s 231us/step - loss: 0.0776 - acc: 0.9626
Epoch 22/50
1123/1123 [==============================] - 0s 227us/step - loss: 0.0798 - acc: 0.9653
Epoch 23/50
1123/1123 [==============================] - 0s 226us/step - loss: 0.0728 - acc: 0.9679 0s - loss: 0.0764 - acc: 0.96
Epoch 24/50
1123/1123 [==============================] - 0s 231us/step - loss: 0.0664 - acc: 0.9697
Epoch 25/50
1123/1123 [==============================] - 0s 227us/step - loss: 0.0684 - acc: 0.9671
Epoch 26/50
1123/1123 [==============================] - 0s 224us/step - loss: 0.0644 - acc: 0.9706
Epoch 27/50
1123/1123 [==============================] - 0s 226us/step - loss: 0.0669 - acc: 0.9688 0s - loss: 0.0703 - acc: 0.9
Epoch 28/50
1123/1123 [==============================] - 0s 233us/step - loss: 0.0673 - acc: 0.9671
Epoch 29/50
1123/1123 [==============================

...

1123/1123 [==============================] - 0s 270us/step - loss: 0.0261 - acc: 0.9902
Epoch 21/50
1123/1123 [==============================] - 0s 280us/step - loss: 0.0113 - acc: 0.9955
Epoch 22/50
1123/1123 [==============================] - 0s 267us/step - loss: 0.0097 - acc: 0.9955
Epoch 23/50
1123/1123 [==============================] - 0s 271us/step - loss: 0.0092 - acc: 0.9973
Epoch 24/50
1123/1123 [==============================] - 0s 267us/step - loss: 0.0095 - acc: 0.9955
Epoch 25/50
1123/1123 [==============================] - 0s 269us/step - loss: 0.0074 - acc: 0.9982 0s - loss: 0.0023 - acc: 1
Epoch 26/50
1123/1123 [==============================] - 0s 267us/step - loss: 0.0128 - acc: 0.9964
Epoch 27/50
1123/1123 [==============================] - 0s 272us/step - loss: 0.0202 - acc: 0.9938
Epoch 28/50
1123/1123 [==============================] - 0s 272us/step - loss: 0.0090 - acc: 0.9964
Epoch 29/50
1123/1123 [==============================] - 0s 276us/step - loss: 0.0090

1123/1123 [==============================] - 0s 268us/step - loss: 8.8787e-06 - acc: 1.0000
Epoch 50/50
1123/1123 [==============================] - 0s 268us/step - loss: 1.6744e-04 - acc: 1.0000
Training of node 47 complete


Training chain node  48
Shapes:
 X = (1123, 1049) 
 Y = (1123, 1)
Epoch 1/50
1123/1123 [==============================] - 3s 2ms/step - loss: 0.1990 - acc: 0.9742
Epoch 2/50
1123/1123 [==============================] - 0s 278us/step - loss: 0.1451 - acc: 0.9893
Epoch 3/50
1123/1123 [==============================] - 0s 268us/step - loss: 0.0992 - acc: 0.9884
Epoch 4/50
1123/1123 [==============================] - 0s 268us/step - loss: 0.0866 - acc: 0.9875
Epoch 5/50
1123/1123 [==============================] - 0s 283us/step - loss: 0.0603 - acc: 0.9875
Epoch 6/50
1123/1123 [==============================] - 0s 265us/step - loss: 0.0652 - acc: 0.9929
Epoch 7/50
1123/1123 [==============================] - 0s 264us/step - loss: 0.0624 - acc: 0.9893
Epoch 8/50
1123/

1123/1123 [==============================] - 0s 265us/step - loss: 0.0395 - acc: 0.9849
Epoch 29/50
1123/1123 [==============================] - 0s 265us/step - loss: 0.0410 - acc: 0.9804
Epoch 30/50
1123/1123 [==============================] - 0s 264us/step - loss: 0.0316 - acc: 0.9822
Epoch 31/50
1123/1123 [==============================] - 0s 267us/step - loss: 0.0576 - acc: 0.9777
Epoch 32/50
1123/1123 [==============================] - 0s 265us/step - loss: 0.0433 - acc: 0.9768
Epoch 33/50
1123/1123 [==============================] - 0s 265us/step - loss: 0.0348 - acc: 0.9813
Epoch 34/50
1123/1123 [==============================] - 0s 266us/step - loss: 0.0320 - acc: 0.9849
Epoch 35/50
1123/1123 [==============================] - 0s 266us/step - loss: 0.0364 - acc: 0.9813
Epoch 36/50
1123/1123 [==============================] - 0s 261us/step - loss: 0.0402 - acc: 0.9822
Epoch 37/50
1123/1123 [==============================] - 0s 258us/step - loss: 0.0336 - acc: 0.9813
Epoch 38/50


1123/1123 [==============================] - 0s 271us/step - loss: 0.0054 - acc: 0.9964
Epoch 9/50
1123/1123 [==============================] - 0s 271us/step - loss: 0.0052 - acc: 0.9964
Epoch 10/50
1123/1123 [==============================] - 0s 266us/step - loss: 0.0041 - acc: 0.9964
Epoch 11/50
1123/1123 [==============================] - 0s 264us/step - loss: 0.0056 - acc: 0.9964
Epoch 12/50
1123/1123 [==============================] - 0s 269us/step - loss: 0.0050 - acc: 0.9955
Epoch 13/50
1123/1123 [==============================] - 0s 266us/step - loss: 0.0045 - acc: 0.9964
Epoch 14/50
1123/1123 [==============================] - 0s 263us/step - loss: 0.0046 - acc: 0.9964
Epoch 15/50
1123/1123 [==============================] - 0s 272us/step - loss: 0.0032 - acc: 0.9964
Epoch 16/50
1123/1123 [==============================] - 0s 273us/step - loss: 0.0033 - acc: 0.9982
Epoch 17/50
1123/1123 [==============================] - 0s 266us/step - loss: 0.0035 - acc: 0.9982
Epoch 18/50
1

1123/1123 [==============================] - 0s 270us/step - loss: 0.0144 - acc: 0.9991
Epoch 40/50
1123/1123 [==============================] - 0s 272us/step - loss: 0.0144 - acc: 0.9991
Epoch 41/50
1123/1123 [==============================] - 0s 273us/step - loss: 0.0144 - acc: 0.9991
Epoch 42/50
1123/1123 [==============================] - 0s 271us/step - loss: 0.0144 - acc: 0.9991
Epoch 43/50
1123/1123 [==============================] - 0s 266us/step - loss: 0.0144 - acc: 0.9991
Epoch 44/50
1123/1123 [==============================] - 0s 274us/step - loss: 0.0144 - acc: 0.9991
Epoch 45/50
1123/1123 [==============================] - 0s 271us/step - loss: 0.0144 - acc: 0.9991
Epoch 46/50
1123/1123 [==============================] - 0s 268us/step - loss: 0.0144 - acc: 0.9991
Epoch 47/50
1123/1123 [==============================] - 0s 271us/step - loss: 0.0144 - acc: 0.9991
Epoch 48/50
1123/1123 [==============================] - 0s 274us/step - loss: 0.0144 - acc: 0.9991
Epoch 49/50


In [9]:
#Model chain
y_pred = []

X_feed = X_test.copy()
i = 0
for node in chain:
    print("Getting op from chain node ",i)
    node_name = "Node" + str(i)
    print("Shapes:\n X = {}".format(X_feed.shape))
    output = node.predict(X_feed)
    output = output.round().astype(int)
    y_pred.append(output)
    X_feed = np.append(X_feed,output,axis=1)
    i+=1
    #Updating X_feed
            

Getting op from chain node  0
Shapes:
 X = (579, 1001)
Getting op from chain node  1
Shapes:
 X = (579, 1002)
Getting op from chain node  2
Shapes:
 X = (579, 1003)
Getting op from chain node  3
Shapes:
 X = (579, 1004)
Getting op from chain node  4
Shapes:
 X = (579, 1005)
Getting op from chain node  5
Shapes:
 X = (579, 1006)
Getting op from chain node  6
Shapes:
 X = (579, 1007)
Getting op from chain node  7
Shapes:
 X = (579, 1008)
Getting op from chain node  8
Shapes:
 X = (579, 1009)
Getting op from chain node  9
Shapes:
 X = (579, 1010)
Getting op from chain node  10
Shapes:
 X = (579, 1011)
Getting op from chain node  11
Shapes:
 X = (579, 1012)
Getting op from chain node  12
Shapes:
 X = (579, 1013)
Getting op from chain node  13
Shapes:
 X = (579, 1014)
Getting op from chain node  14
Shapes:
 X = (579, 1015)
Getting op from chain node  15
Shapes:
 X = (579, 1016)
Getting op from chain node  16
Shapes:
 X = (579, 1017)
Getting op from chain node  17
Shapes:
 X = (579, 1018)
Ge

In [10]:
predictions = np.array([0])
for i in y_pred:
    x = np.array(i)
    if predictions.shape == (1,):
        predictions = x
    else:
        predictions = np.append(predictions,x,axis=1)

In [11]:
# accuracy
from sklearn.metrics import hamming_loss, log_loss, f1_score,accuracy_score
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("Hamming accuracy = ",hamming_accuracy_score(y_test,predictions))
print("Hamming loss = ",hamming_loss(y_test,predictions))
print("Log loss = ",log_loss(y_test,predictions))
print("F1 score = ",f1_score(y_test,predictions,average='macro'))

Accuracy =  0.155440414508
Hamming accuracy =  0.448512009004
Hamming loss =  0.05122690390067455
Log loss =  60.5806239444
F1 score =  0.20066038712
